In [27]:
import os
directory = r'D:\github\localt20\data\processedData'

import polars as pl
matches = pl.read_csv(os.path.join(directory, 'matches.csv'))
team_data = pl.read_csv(os.path.join(directory, 'teamStats.csv'))
matches

team1,team2,gender,season,date,venue,city,toss_winner,toss_decision,winner,match_id
str,str,str,str,str,str,str,str,str,str,i64
"""Australia""","""Sri Lanka""","""male""","""2016/17""","""2017/02/17""","""Melbourne Cricket Ground""",null,"""Sri Lanka""","""field""","""Sri Lanka""",1001349
"""Australia""","""Sri Lanka""","""male""","""2016/17""","""2017/02/19""","""Simonds Stadium, South Geelong""","""Victoria""","""Sri Lanka""","""field""","""Sri Lanka""",1001351
"""Australia""","""Sri Lanka""","""male""","""2016/17""","""2017/02/22""","""Adelaide Oval""",null,"""Sri Lanka""","""field""","""Australia""",1001353
"""Ireland""","""Hong Kong""","""male""","""2016""","""2016/09/05""","""Bready Cricket Club, Magherama…","""Londonderry""","""Hong Kong""","""bat""","""Hong Kong""",1004729
"""Zimbabwe""","""India""","""male""","""2016""","""2016/06/18""","""Harare Sports Club""",null,"""India""","""field""","""Zimbabwe""",1007655
…,…,…,…,…,…,…,…,…,…,…
"""Pakistan""","""Sri Lanka""","""male""","""2015/16""","""2016/03/04""","""Shere Bangla National Stadium""","""Mirpur""","""Pakistan""","""field""","""Pakistan""",966763
"""Bangladesh""","""India""","""male""","""2015/16""","""2016/03/06""","""Shere Bangla National Stadium""","""Mirpur""","""India""","""field""","""India""",966765
"""United Arab Emirates""","""Netherlands""","""male""","""2015/16""","""2016/02/03""","""ICC Academy""","""Dubai""","""United Arab Emirates""","""field""","""Netherlands""",967081


In [28]:
tdt = team_data['Team'].unique().to_list()
mdt = matches['team1'].unique().to_list()

In [29]:
for i in tdt:
    if i not in mdt:
        print(i)

Czech Rep.
U.S.A.
U.A.E.
ICC World XI
Peru
World-XI
Cayman
P.N.G.


In [30]:
for i in mdt:
    if i not in tdt:
        print(i)

United Arab Emirates
Czech Republic
United States of America
Cayman Islands
Barbados
Papua New Guinea


In [31]:
# Teams that need to be mapped between `tdt` and `mdt`
team_name_mapping = {
    'U.S.A.': 'United States of America',
    'U.A.E.': 'United Arab Emirates',
    'Czech Rep.': 'Czech Republic',
    'P.N.G.': 'Papua New Guinea',
    'Cayman': 'Cayman Islands'
}

# Teams that do not have a direct match
unmatched_tdt = [team for team in tdt if team not in mdt and team not in team_name_mapping]
unmatched_mdt = [team for team in mdt if team not in tdt and team not in team_name_mapping.values()]

print("Mapped Team Names Dictionary:", team_name_mapping)
print("Unmatched Teams in tdt:", unmatched_tdt)
print("Unmatched Teams in mdt:", unmatched_mdt)

Mapped Team Names Dictionary: {'U.S.A.': 'United States of America', 'U.A.E.': 'United Arab Emirates', 'Czech Rep.': 'Czech Republic', 'P.N.G.': 'Papua New Guinea', 'Cayman': 'Cayman Islands'}
Unmatched Teams in tdt: ['ICC World XI', 'Peru', 'World-XI']
Unmatched Teams in mdt: ['Barbados']


In [32]:
unmatched_teams = unmatched_tdt + unmatched_mdt
unmatched_teams

['ICC World XI', 'Peru', 'World-XI', 'Barbados']

In [33]:
print(team_data.shape, matches.shape)
team_data = team_data.filter(~pl.col('Team').is_in(unmatched_teams)).sort('Team')
matches = matches.filter(~pl.col('team1').is_in(unmatched_teams)).sort('team1')
matches = matches.filter(~pl.col('team2').is_in(unmatched_teams)).sort('team2')
print(team_data.shape, matches.shape)

(907, 13) (3683, 11)
(904, 13) (3676, 11)


In [34]:
team_name_mapping

{'U.S.A.': 'United States of America',
 'U.A.E.': 'United Arab Emirates',
 'Czech Rep.': 'Czech Republic',
 'P.N.G.': 'Papua New Guinea',
 'Cayman': 'Cayman Islands'}

In [35]:
team_data = team_data.with_columns(pl.col('Team').map_elements(lambda x: team_name_mapping.get(x, x),return_dtype=pl.String).alias('Team'))
matches = matches.with_columns(pl.col('team1').map_elements(lambda x: team_name_mapping.get(x, x),return_dtype=pl.String).alias('team1'))
matches = matches.with_columns(pl.col('team2').map_elements(lambda x: team_name_mapping.get(x, x),return_dtype=pl.String).alias('team2'))
matches

team1,team2,gender,season,date,venue,city,toss_winner,toss_decision,winner,match_id
str,str,str,str,str,str,str,str,str,str,i64
"""Australia""","""Afghanistan""","""male""","""2022/23""","""2022/11/04""","""Adelaide Oval""","""Adelaide""","""Afghanistan""","""field""","""Australia""",1298172
"""Bangladesh""","""Afghanistan""","""male""","""2018""","""2018/06/05""","""Rajiv Gandhi International Cri…","""Dehradun""","""Bangladesh""","""bat""","""Afghanistan""",1145983
"""Bangladesh""","""Afghanistan""","""male""","""2019""","""2019/09/15""","""Shere Bangla National Stadium,…","""Dhaka""","""Afghanistan""","""bat""","""Afghanistan""",1197142
"""Bangladesh""","""Afghanistan""","""male""","""2019""","""2019/09/21""","""Zahur Ahmed Chowdhury Stadium""","""Chattogram""","""Bangladesh""","""field""","""Bangladesh""",1197145
"""Bangladesh""","""Afghanistan""","""male""","""2021/22""","""2022/03/03""","""Shere Bangla National Stadium,…","""Dhaka""","""Bangladesh""","""bat""","""Bangladesh""",1299832
…,…,…,…,…,…,…,…,…,…,…
"""United Arab Emirates""","""Zimbabwe""","""male""","""2013/14""","""2014/03/21""","""Sylhet Stadium""",null,"""Zimbabwe""","""field""","""Zimbabwe""",682917
"""West Indies""","""Zimbabwe""","""male""","""2022/23""","""2022/10/19""","""Bellerive Oval, Hobart""","""Hobart""","""West Indies""","""bat""","""West Indies""",1298142
"""West Indies""","""Zimbabwe""","""male""","""2009/10""","""2010/02/28""","""Queen's Park Oval, Port of Spa…","""Trinidad""","""Zimbabwe""","""bat""","""Zimbabwe""",439139


# Flipping

In [36]:
matches1 = matches
matches2 = matches.rename({'team1': 'team2', 'team2': 'team1'}).select(['team1','team2','gender','season','date','venue','city','toss_winner','toss_decision','winner','match_id'])
matchesflip = pl.concat([matches1, matches2]).sort('match_id')
matchesflip

team1,team2,gender,season,date,venue,city,toss_winner,toss_decision,winner,match_id
str,str,str,str,str,str,str,str,str,str,i64
"""England""","""Australia""","""male""","""2005""","""2005/06/13""","""The Rose Bowl""","""Southampton""","""England""","""bat""","""England""",211028
"""Australia""","""England""","""male""","""2005""","""2005/06/13""","""The Rose Bowl""","""Southampton""","""England""","""bat""","""England""",211028
"""New Zealand""","""Australia""","""male""","""2004/05""","""2005/02/17""","""Eden Park""","""Auckland""","""Australia""","""bat""","""Australia""",211048
"""Australia""","""New Zealand""","""male""","""2004/05""","""2005/02/17""","""Eden Park""","""Auckland""","""Australia""","""bat""","""Australia""",211048
"""South Africa""","""New Zealand""","""male""","""2005/06""","""2005/10/21""","""New Wanderers Stadium""","""Johannesburg""","""New Zealand""","""field""","""New Zealand""",222678
…,…,…,…,…,…,…,…,…,…,…
"""Spain""","""Greece""","""female""","""2024""","""2024/09/21""","""Marina Ground, Corfu""","""Corfu""","""Spain""","""field""","""Spain""",1451292
"""Greece""","""Spain""","""female""","""2024""","""2024/09/22""","""Marina Ground, Corfu""","""Corfu""","""Spain""","""field""","""Spain""",1451293
"""Spain""","""Greece""","""female""","""2024""","""2024/09/22""","""Marina Ground, Corfu""","""Corfu""","""Spain""","""field""","""Spain""",1451293


In [37]:
matches.null_count()

team1,team2,gender,season,date,venue,city,toss_winner,toss_decision,winner,match_id
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,192,0,0,0,0


In [38]:
data = matchesflip.join(team_data, left_on=['team1','season'], right_on=['Team','Season'], how='inner',suffix='_team1')
data

team1,team2,gender,season,date,venue,city,toss_winner,toss_decision,winner,match_id,Mat,Won,Lost,Tied,NR,W/L,Inns,HS,LS,AveRPW,AveRPO
str,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,f64,i64,i64,f64,f64,f64
"""England""","""Australia""","""male""","""2005""","""2005/06/13""","""The Rose Bowl""","""Southampton""","""England""","""bat""","""England""",211028,1,1,0,0,0,1.0,1,179,0.0,22.37,8.95
"""Australia""","""England""","""male""","""2005""","""2005/06/13""","""The Rose Bowl""","""Southampton""","""England""","""bat""","""England""",211028,1,0,1,0,0,0.0,1,79,79.0,7.9,5.44
"""New Zealand""","""Australia""","""male""","""2004/05""","""2005/02/17""","""Eden Park""","""Auckland""","""Australia""","""bat""","""Australia""",211048,1,0,1,0,0,0.0,1,170,170.0,17.0,8.5
"""Australia""","""New Zealand""","""male""","""2004/05""","""2005/02/17""","""Eden Park""","""Auckland""","""Australia""","""bat""","""Australia""",211048,1,1,0,0,0,1.0,1,214,0.0,42.8,10.7
"""South Africa""","""New Zealand""","""male""","""2005/06""","""2005/10/21""","""New Wanderers Stadium""","""Johannesburg""","""New Zealand""","""field""","""New Zealand""",222678,3,1,2,0,0,0.5,3,201,114.0,18.66,7.72
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Spain""","""Greece""","""female""","""2024""","""2024/09/21""","""Marina Ground, Corfu""","""Corfu""","""Spain""","""field""","""Spain""",1451292,11,11,0,0,0,11.0,11,230,0.0,27.3,8.82
"""Greece""","""Spain""","""female""","""2024""","""2024/09/22""","""Marina Ground, Corfu""","""Corfu""","""Spain""","""field""","""Spain""",1451293,4,0,4,0,0,0.0,4,134,84.0,11.5,5.42
"""Spain""","""Greece""","""female""","""2024""","""2024/09/22""","""Marina Ground, Corfu""","""Corfu""","""Spain""","""field""","""Spain""",1451293,11,11,0,0,0,11.0,11,230,0.0,27.3,8.82


In [39]:
data = data.join(team_data, left_on=['team2','season'], right_on=['Team','Season'], how='inner',suffix='_team2')
data

team1,team2,gender,season,date,venue,city,toss_winner,toss_decision,winner,match_id,Mat,Won,Lost,Tied,NR,W/L,Inns,HS,LS,AveRPW,AveRPO,Mat_team2,Won_team2,Lost_team2,Tied_team2,NR_team2,W/L_team2,Inns_team2,HS_team2,LS_team2,AveRPW_team2,AveRPO_team2
str,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,f64,i64,i64,f64,f64,f64,i64,i64,i64,i64,i64,f64,i64,i64,f64,f64,f64
"""England""","""Australia""","""male""","""2005""","""2005/06/13""","""The Rose Bowl""","""Southampton""","""England""","""bat""","""England""",211028,1,1,0,0,0,1.0,1,179,0.0,22.37,8.95,1,0,1,0,0,0.0,1,79,79.0,7.9,5.44
"""Australia""","""England""","""male""","""2005""","""2005/06/13""","""The Rose Bowl""","""Southampton""","""England""","""bat""","""England""",211028,1,0,1,0,0,0.0,1,79,79.0,7.9,5.44,1,1,0,0,0,1.0,1,179,0.0,22.37,8.95
"""New Zealand""","""Australia""","""male""","""2004/05""","""2005/02/17""","""Eden Park""","""Auckland""","""Australia""","""bat""","""Australia""",211048,1,0,1,0,0,0.0,1,170,170.0,17.0,8.5,1,1,0,0,0,1.0,1,214,0.0,42.8,10.7
"""Australia""","""New Zealand""","""male""","""2004/05""","""2005/02/17""","""Eden Park""","""Auckland""","""Australia""","""bat""","""Australia""",211048,1,1,0,0,0,1.0,1,214,0.0,42.8,10.7,1,0,1,0,0,0.0,1,170,170.0,17.0,8.5
"""South Africa""","""New Zealand""","""male""","""2005/06""","""2005/10/21""","""New Wanderers Stadium""","""Johannesburg""","""New Zealand""","""field""","""New Zealand""",222678,3,1,2,0,0,0.5,3,201,114.0,18.66,7.72,2,1,0,1,0,1.0,2,134,0.0,20.0,6.84
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Spain""","""Greece""","""female""","""2024""","""2024/09/21""","""Marina Ground, Corfu""","""Corfu""","""Spain""","""field""","""Spain""",1451292,11,11,0,0,0,11.0,11,230,0.0,27.3,8.82,4,0,4,0,0,0.0,4,134,84.0,11.5,5.42
"""Greece""","""Spain""","""female""","""2024""","""2024/09/22""","""Marina Ground, Corfu""","""Corfu""","""Spain""","""field""","""Spain""",1451293,4,0,4,0,0,0.0,4,134,84.0,11.5,5.42,11,11,0,0,0,11.0,11,230,0.0,27.3,8.82
"""Spain""","""Greece""","""female""","""2024""","""2024/09/22""","""Marina Ground, Corfu""","""Corfu""","""Spain""","""field""","""Spain""",1451293,11,11,0,0,0,11.0,11,230,0.0,27.3,8.82,4,0,4,0,0,0.0,4,134,84.0,11.5,5.42


In [40]:
data = data.with_columns(pl.when(pl.col("gender")=="male").then(1).otherwise(0).alias('gender'))
data.head()

team1,team2,gender,season,date,venue,city,toss_winner,toss_decision,winner,match_id,Mat,Won,Lost,Tied,NR,W/L,Inns,HS,LS,AveRPW,AveRPO,Mat_team2,Won_team2,Lost_team2,Tied_team2,NR_team2,W/L_team2,Inns_team2,HS_team2,LS_team2,AveRPW_team2,AveRPO_team2
str,str,i32,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,f64,i64,i64,f64,f64,f64,i64,i64,i64,i64,i64,f64,i64,i64,f64,f64,f64
"""England""","""Australia""",1,"""2005""","""2005/06/13""","""The Rose Bowl""","""Southampton""","""England""","""bat""","""England""",211028,1,1,0,0,0,1.0,1,179,0.0,22.37,8.95,1,0,1,0,0,0.0,1,79,79.0,7.9,5.44
"""Australia""","""England""",1,"""2005""","""2005/06/13""","""The Rose Bowl""","""Southampton""","""England""","""bat""","""England""",211028,1,0,1,0,0,0.0,1,79,79.0,7.9,5.44,1,1,0,0,0,1.0,1,179,0.0,22.37,8.95
"""New Zealand""","""Australia""",1,"""2004/05""","""2005/02/17""","""Eden Park""","""Auckland""","""Australia""","""bat""","""Australia""",211048,1,0,1,0,0,0.0,1,170,170.0,17.0,8.5,1,1,0,0,0,1.0,1,214,0.0,42.8,10.7
"""Australia""","""New Zealand""",1,"""2004/05""","""2005/02/17""","""Eden Park""","""Auckland""","""Australia""","""bat""","""Australia""",211048,1,1,0,0,0,1.0,1,214,0.0,42.8,10.7,1,0,1,0,0,0.0,1,170,170.0,17.0,8.5
"""South Africa""","""New Zealand""",1,"""2005/06""","""2005/10/21""","""New Wanderers Stadium""","""Johannesburg""","""New Zealand""","""field""","""New Zealand""",222678,3,1,2,0,0,0.5,3,201,114.0,18.66,7.72,2,1,0,1,0,1.0,2,134,0.0,20.0,6.84


In [41]:
# gender	match_id	Mat	Won	Lost	Tied	NR	W/L	Inns	HS	LS	AveRPW	AveRPO	Mat_team2	Won_team2	Lost_team2	Tied_team2	NR_team2	W/L_team2	Inns_team2	HS_team2	LS_team2	AveRPW_team2	AveRPO_team2
data = data.drop('team1','team2','season','date','venue','city','toss_winner','toss_decision','winner').select(['match_id','gender','Mat','Won','Lost','Tied','NR','W/L','Inns','HS','LS','AveRPW','AveRPO','Mat_team2','Won_team2','Lost_team2','Tied_team2','NR_team2','W/L_team2','Inns_team2','HS_team2','LS_team2','AveRPW_team2','AveRPO_team2'])
data

match_id,gender,Mat,Won,Lost,Tied,NR,W/L,Inns,HS,LS,AveRPW,AveRPO,Mat_team2,Won_team2,Lost_team2,Tied_team2,NR_team2,W/L_team2,Inns_team2,HS_team2,LS_team2,AveRPW_team2,AveRPO_team2
i64,i32,i64,i64,i64,i64,i64,f64,i64,i64,f64,f64,f64,i64,i64,i64,i64,i64,f64,i64,i64,f64,f64,f64
211028,1,1,1,0,0,0,1.0,1,179,0.0,22.37,8.95,1,0,1,0,0,0.0,1,79,79.0,7.9,5.44
211028,1,1,0,1,0,0,0.0,1,79,79.0,7.9,5.44,1,1,0,0,0,1.0,1,179,0.0,22.37,8.95
211048,1,1,0,1,0,0,0.0,1,170,170.0,17.0,8.5,1,1,0,0,0,1.0,1,214,0.0,42.8,10.7
211048,1,1,1,0,0,0,1.0,1,214,0.0,42.8,10.7,1,0,1,0,0,0.0,1,170,170.0,17.0,8.5
222678,1,3,1,2,0,0,0.5,3,201,114.0,18.66,7.72,2,1,0,1,0,1.0,2,134,0.0,20.0,6.84
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1451292,0,11,11,0,0,0,11.0,11,230,0.0,27.3,8.82,4,0,4,0,0,0.0,4,134,84.0,11.5,5.42
1451293,0,4,0,4,0,0,0.0,4,134,84.0,11.5,5.42,11,11,0,0,0,11.0,11,230,0.0,27.3,8.82
1451293,0,11,11,0,0,0,11.0,11,230,0.0,27.3,8.82,4,0,4,0,0,0.0,4,134,84.0,11.5,5.42


In [44]:
import numpy as np
directory = r'D:\github\localt20\data\aftermerging'

data.write_csv(os.path.join(directory, 'team12Statsflip.csv'))